In [4]:
#manipulate/organize data/visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#preprocess data
import unicodedata as ud
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
#Linear regression to fill missing data for certain types
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

In [377]:
valid_types

{'APPART A VENDR',
 'DUPLEX A VENDR',
 'LOFT A VENDR',
 'MAISON A VENDR',
 'MAISON VILL A VENDR',
 'STUDIO A VENDR',
 'TERRAIN CONSTRUCTIBL A VENDR',
 'VILL A VENDR'}

In [477]:
STEM_TO_POSTCODES = {
    "BOULOGN": "92000",                       # Boulogne-Billancourt
    "BILLANCOURT": "92000",                   # Boulogne-Billancourt
    "BOULOGN BILLANCOURT": "92000",           # Boulogne-Billancourt
    "NEUILLY": "92000",                       # Neuilly-sur-Seine
    "NEUILLY SEIN": "92000",                  # Neuilly-sur-Seine
    "LEVALLOIS PERRET": "92000",              # Levallois-Perret
    "CLICHY": "92000",                        # Clichy
    "CLICHY GAREN": "92000",                  # Clichy-la-Garenne (same CP)
    "SAINT CLOUD": "92000",                   # Saint-Cloud
    "CLOUD": "92000",                         # Saint-Cloud
    "PUTEAU": "92000",                        # Puteaux
    "SURESN": "92000",                        # Suresnes
    "ISSY": "92000",                          # Issy-les-Moulineaux
    "ISSY MOULINEAU": "92000",                # Issy-les-Moulineaux
    "MOULINEAU": "92000",                     # Issy-les-Moulineaux
    "MONTROUG": "92000",                      # Montrouge
    "LE LIL": "93000",                        # Les Lilas
    "AUBERVILLI": "93000",                    # Aubervilliers
    "SAINT OUEN SEIN": "93000",               # Saint-Ouen-sur-Seine
    "OUEN": "93000",                          # Saint-Ouen-sur-Seine
    "CHARENTON": "94000",                     # Charenton-le-Pont
    "CHARENTON PONT" : "94000",
    "VANV": "92000",                          # Vanves
    "MONTREUIL": "93000",                     # Montreuil
    "PANTIN": "93000",                        # Pantin
    "PONT": "94000",                          # (mapped by you to Charenton-le-Pont)
    "SEIN": "92000",                          # (mapped by you to Neuilly-sur-Seine)
    "SAINT DEN" : "93000",
    "DEN": "93000",                           # Saint-Denis
    "PARIS": "75000",
    "BAGNOLET": "93000"
}
valid_tokens = {'APPART A VENDR',
     'AUBERVILLI',
     'BAGNOLET',
     'BOULOGN BILLANCOURT',
     'CHAMBR',
     'CHARENTON PONT',
     'CLICHY',
     'CLICHY GAREN',
     'DISPONIBL MAINTEN',
     'DIVISIBL',
     'DUPLEX A VENDR',
     'ETAG',
     'HOTEL PARTICULI A VENDR',
     'ISSY MOULINEAU',
     'LE LIL',
     'LEVALLOIS PERRET',
     'LOFT A VENDR',
     'MAISON A VENDR',
     'MAISON VILL A VENDR',
     'MONTREUIL',
     'MONTROUG',
     'M²',
     'NEUF',
     'NEUILLY SEIN',
     'PANTIN',
     'PARIS',
     'PIEC',
     'PUTEAU',
     'RDC',
     'SAINT CLOUD',
     'SAINT DEN',
     'SAINT OUEN SEIN',
     'STUDIO A VENDR',
     'SURESN',
     'TERRAIN',
     'TERRAIN CONSTRUCTIBL A VENDR',
     'VANV',
     'VILL A VENDR'}
valid_types = {
    'APPART A VENDR',
    'DUPLEX A VENDR',
    'LOFT A VENDR',
    'MAISON A VENDR',
    'MAISON VILL A VENDR',
    'STUDIO A VENDR',
    'TERRAIN CONSTRUCTIBL A VENDR',
    'VILL A VENDR'
}

# Currency tokens (symbol or word forms)
EUR_PATTERN = re.compile(
    r"(€|\b(?:eur|euro(?:s)?)\b)", re.IGNORECASE
)
# ✅ Matches: "€", "EUR", "euro", "euros", "Prix: 200 eur"
# ❌ Doesn’t: "eurostar" (word boundary blocks it), "EUROPE" (not 'euro' exactly)
# ⚠️ Note: Won’t catch "€1,200" as a whole—this only finds the currency token, not the number.

GBP_PATTERN = re.compile(
    r"(£|\b(?:gbp|pound(?:s)?)\b)", re.IGNORECASE
)
# ✅ Matches: "£", "GBP", "pound", "pounds", "200 gbp"
# ❌ Doesn’t: "pounding", "compound" (word boundary prevents false positives)
# ⚠️ "$" in "CA$" is not relevant here; that’s for USD_PATTERN.

USD_PATTERN = re.compile(
    r"(\$|\b(?:usd|dollar(?:s)?)\b)", re.IGNORECASE
)
# ✅ Matches: "$", "USD", "dollar", "dollars", "2 usd"
# ❌ Doesn’t: "sandollar" (needs word boundary), other currency symbols like "C$", "A$"
# ⚠️ "$" is ambiguous across countries; consider context if you need only US dollars.

# General number with optional decimal (dot or comma)
NUM_PATTERN = re.compile(r"\d+(?:[.,]\d+)?")
# ✅ Matches: "5", "12", "83.5", "1,75", "0003"
# ❌ Doesn’t: signed numbers like "-3.2" (see PATTERN_NUM), fractions like "1/7" (you said slash handled separately)
# ⚠️ Will also match date parts ("12/10") individually.

# French ordinal like "1ER", "12EM"
ORDINAL_PATTERN = re.compile(r"\b\d+(?:ER|EM)\b", flags=re.IGNORECASE)
# ✅ Matches: "1ER", "2er", "12EM", "3em"
# ❌ Doesn’t: "1ÈRE", "2ÈME" (accented forms), "1st", "2nd"
# ⚠️ If you need accented French ordinals, expand to include ÈRE/ÈME variants.

# Exact "PARIS" (uppercase only)
PARIS_PATTERN = re.compile(r"PARIS")
# ✅ Matches: "PARIS"
# ❌ Doesn’t: "Paris", "paris" (case-sensitive), arrondissements like "75015 Paris" unless uppercased
# 💡 Consider re.compile(r"\bparis\b", re.I) if you want case-insensitive.

# Signed integer/float with dot decimal
PATTERN_NUM = re.compile(r"[+-]?\d+(?:\.\d+)?")
# ✅ Matches: "-12", "+3.5", "0.99"
# ❌ Doesn’t: comma decimals ("1,75"), fractions ("1/7")
# ⚠️ This one is stricter than NUM_PATTERN (only dot decimals, allows sign).

# Area cues: "m²", "m2", "surface…", or "terrain"
PATTERN_AREA = re.compile(r"(?:[-–—]?\s*)(?:M(?:²|2)|SURFAC\w*|TERRAIN)", re.I)
# ✅ Matches: "m²", "M2", "surface", "surfaces", "terrain", with optional preceding dash/space
# ❌ Doesn’t: "m^2" (caret), "area" (English), "superficie" (if that term appears, it’s missed)
# ⚠️ "TERRAIN" here might be too broad if you treat lots vs. living area differently.

# Standalone "TERRAIN"
PATTERN_TERRAIN = re.compile(r"\bTERRAIN\b", re.I)
# ✅ Matches only the word "terrain" as a token
# ❌ Doesn’t: "terrainment", "souterrain" (good—no false positives)

# Any digit anywhere
PATTERN_HAS_DIGIT = re.compile(r"\d")
# ✅ Matches if string contains at least one digit
# ❌ Doesn’t: strings with numbers written as words ("trois", "three")

# Availability "disponible maintenant" with flexible endings
PATTERN_DISPO_NOW = re.compile(r"\bDISPONIBL\w*\s+MAINTEN\w*\b", re.I)
# ✅ Matches: "disponible maintenant", "Disponibilité maintenue" (careful), "disponibles maintien…"
# ❌ Doesn’t: rearranged word order ("maintenant disponible"), abbreviations
# ⚠️ Broad \w* can overmatch ("maintenue"); tighten if needed to specifically "disponible maintenant".

# Floor indicators: "étage" or "RDC" (ground floor)
PATTERN_FLOOR = re.compile(r"\b(?:ETAG\w*|RDC)\b", re.I)
# ✅ Matches: "etage", "étages", "RDC"
# ❌ Doesn’t: "Rez-de-chaussée" written fully (unless you rely on "RDC")
# ⚠️ You also have dedicated ETAG/RDC patterns below; avoid double work.
    
PATTERN_ETAG = re.compile(r"\bETAG\w*\b", re.I)
# ✅ Matches: "etage", "étages", "étagé" (could be false positive)
# ❌ Doesn’t: "Rez-de-chaussée", "R+1" (common shorthand not covered)

PATTERN_RDC = re.compile(r"\bRDC\w*\b", re.I)
# ✅ Matches: "RDC", "RDC+1" (because \w*), "RDC."
# ❌ Doesn’t: "Rez de chaussée" written without "RDC"
# ⚠️ If you don’t want "RDC+1", use r"\bRDC\b".

# Rooms / pieces
PATTERN_PIECE = re.compile(r"\bPIEC\w*\b", re.I)
# ✅ Matches: "piece", "pièces", "pièce(s)"
# ❌ Doesn’t: abbreviations like "T2", "F3" (common in FR real-estate)

# New / newly built
PATTERN_NEUF = re.compile(r"\bNEUF\w*\b", re.I)
# ✅ Matches: "neuf", "neuve", "neufs", "neuves"
# ❌ Doesn’t: "neuf" meaning the number 9 when used numerically (context needed)
# ⚠️ Ambiguous: "neuf" can be adjective "new" or the noun/adjective "nine".

# Bedrooms
PATTERN_CHAMBRE = re.compile(r"\bCHAMBR\w*\b", re.I)
# ✅ Matches: "chambre", "chambres"
# ❌ Doesn’t: abbreviations ("chb", "CH."), "suite parentale" (no keyword "chambre")

# Number BEFORE a slash (capture group = the number)
PATTERN_BEFORE_SLASH = re.compile(r"([+-]?\d+(?:\.\d+)?)\s*/")
# ✅ Matches the "12" in "12 / 5", "+3.5/7"
# ❌ Doesn’t: "12,5/7" (comma decimals), no slash present
# ⚠️ Will also match dates ("12/10")—disambiguate upstream if needed.

# Number AFTER a slash (capture group = the number)
PATTERN_AFTER_SLASH = re.compile(r"/\s*([+-]?\d+(?:\.\d+)?)")
# ✅ Matches the "5" in "12 / 5", the "7" in "3.5/7"
# ❌ Doesn’t: "12/1,5" (comma decimals)

# Literal slash (useful for quick checks/splits)
PATTERN_SLASH = re.compile(r"/")
# ✅ Matches any "/" character
# ❌ Doesn’t: division written as "÷" or "per" words


In [34]:
def remove_accents(text: str):
    #replace diacritics
    text = (text.replace("œ", "oe").replace("Œ", "OE")
           .replace("æ", "ae").replace("Æ", "AE"))
    #decompose form 'è' -> 'e' + '`'
    text = ud.normalize("NFD",text)
    #remove accents like '`','^',...
    text = "".join([char for char in text if ud.category(char) != "Mn"])
    #recompose 
    text = ud.normalize("NFC",text)
    return str(text)

In [36]:
def preprocess_french_nltk(text: str):
    text = remove_accents(text)
    stemmer = SnowballStemmer("french")
    stop_fr = set(stopwords.words("french"))
    tkns = re.findall(r"[A-Za-zÀ-ÖØ-öø-ÿ\d/(m2|m²).]+", text)
    tkns = [t for t in tkns if t not in stop_fr]
    stems  = [stemmer.stem(t) for t in tkns]
    return (" ".join(stems)).upper()

In [38]:
def extract_tokens(text : str = ""):
    #convert "," of float num to "."
    text = re.sub(r"(?<=\d),(?=\d)",".",text)
    #convert "-" to " "
    text = re.sub(r'(?<=[^\W\d_])-(?=[^\W\d_])', ' ', text, flags=re.UNICODE)
    tkns = [tkn.strip() for tkn in re.split("[-,]",text)]
    return tkns

In [40]:
def convert_2price(price_str : str = None)->str:
    try :
        price_str = "".join([chr for chr in price_str if chr.isdigit()])
        return float(price_str)
    except Exception :
        return None

In [42]:
def extract_price(tkns : list = None):
    if not tkns :
        return None
    EUR_PATTERN = re.compile(r"(€|\b(?:eur|euro(?:s)?)\b)", re.IGNORECASE)
    GBP_PATTERN = re.compile(r"(£|\b(?:gbp|pound(?:s)?)\b)", re.IGNORECASE)
    USD_PATTERN = re.compile(r"(\$|\b(?:usd|dollar(?:s)?)\b)", re.IGNORECASE)
    m_tkns = []
    prices = []
    if not tkns :
        raise ValueError("elements = None")
    for tkn in tkns :
        if bool(EUR_PATTERN.search(tkn)):
            prices.append([tkn,"EUR"])
        elif bool(GBP_PATTERN.search(tkn)):
            prices.append([tkn,"GBP"])
        elif bool(USD_PATTERN.search(tkn)):
            prices.append([tkn,"USD"])
        else :
            m_tkns.append(tkn)
    if len(prices) != 1:
        raise ValueError("More than 1 or no price for this property")
    return [m_tkns,prices[0]]

In [208]:
def get_processed(raw_df):
    #df = pd.DataFrame(columns=["_","PRICE","UNIT-PRICE"])
    list_tags = []
    prices =[]
    price_units = []
    raws = []
    for i in range(len(raw_df)):
        tags,(price_str,price_unit)=extract_price(extract_tokens(raw_df.iloc[i]))
        tags = list(map(preprocess_french_nltk,tags))
        price = convert_2price(price_str)
        if price == None :
            print(price_str)
        if price > 1000 :# add threshold to prevent false pricing
            list_tags.append(tags)
            prices.append(price)
            price_units.append(price_unit)
            raws.append(raw_df.iloc[i])
    return (pd.DataFrame({"RAW" : tuple(raws),"DATA TAG":list_tags,"PRICE":prices,"PRICE_UNIT":price_units}))

In [479]:
def normalize_token(tag):
    tag = tag.upper().strip()
    if "ETAG" in tag :
        return "ETAG"
    if 'RDC' in tag:    
        return 'RDC'
    """
    if 'TERRAIN' in tag:
        return 'TERRAIN'
    """
    if 'SURFAC' in tag:
        return 'SURFAC'
    #remove tags
    if 'DIVISIBL A PART M²' in tag or 'DIVISIBL JUSQU A M²' in tag or 'DIVISIBL M² A M²' in tag:
        return None
    #reduce all M² to M²
    if 'M²' in tag :
        return 'M²'

    # Remove ordinals like 1ER / 12EM and plain numbers/decimals
    tag = ORDINAL_PATTERN.sub('', tag)
    tag = tag.replace('/', ' ')            # split fractions like "1/7"
    tag = NUM_PATTERN.sub('', tag)

    # Collapse whitespace
    tag = re.sub(r'\s+', ' ', tag).strip()

    return tag

In [138]:
def normalize_tags(tags):
    n_tkns = [] #normalized tokens
    for tkn in tags :
        if str(tkn).strip() :
            n_tkn = normalize_token(tkn)
            if n_tkn:
                if n_tkn in valid_tokens :
                    n_tkns.append(n_tkn)
                else :
                    return pd.NA
    if len(n_tkns) == 0 :
        return None
    return tuple(n_tkns)

In [108]:
def clean_tags(seqs):
    return {normalize_sequence(tags) for tags in seqs }

In [128]:
def extract_type(cleaned_tags):
    for t in cleaned_tags :
        if t in valid_types :
            return t
    return pd.NA

In [142]:
def get_paris_cp(tags):
    paris_tag = ""
    for tag in tags :
        if "PARIS" in tag :
            if not bool(re.search(r"\d", tag))  :
                print("Got tag with no digit : ", tag)
                return None
            try :
                digits = int("".join([ch for ch in tag if ch.isdigit()]))
                if digits <= 20 and digits > 0:
                    return str(75000 + int(digits))
                else :
                    print("District code doesn't exist")
                    return None
            except Exception :
                print(f"Error encountered (got digits = {digits}): ", tag)
                return None
    return None

In [160]:
#now let's extract all locations to a column
def extract_loc(df,dict_map):
    """
    extract_loc is gonna focus on inside paris, and others will be more generalized 
    """
    df= df.copy()
    mask_not_paris = df["CLEANED TAG"].apply(lambda x: isinstance(x, (list, tuple, set)) and "PARIS" not in x)
    
    valid_locs = set(dict_map.keys())
    df.loc[mask_not_paris, "LOC"] = (
        df.loc[mask_not_paris, "CLEANED TAG"]
          .apply(lambda x: (set(x) & valid_locs) if x else pd.NA)
          .apply(lambda x: {dict_map[c] for c in x} if x else pd.NA)
    )
    
    df.loc[~mask_not_paris,"LOC"] = (
        df[~mask_not_paris]["DATA TAG"].apply(lambda x : set([get_paris_cp(x)]) if get_paris_cp(x) else pd.NA)
    )
    
    mask_multi_loc = df["LOC"].apply(lambda x: isinstance(x, (list, tuple, set)) and len(x) > 1)

    #print("Multi loc assigned detected : ",(mask_multi_loc & mask_not_paris).sum())
    return df

In [164]:
def _numbers_in_text(s):
    """Return all numbers (as floats) found in a string. Spaces already preprocessed upstream."""
    if not isinstance(s, str):
        return []
    s_clean = s.replace(" ", "")  # e.g. '2 645' -> '2645'
    return [float(m.group(0)) for m in PATTERN_NUM.finditer(s_clean)]

In [180]:
"""
To normalize surface area,
we decide to bring all to a singular scalar value for all type of surface except TERRAIN, which will be kept as 2 scalars.
-> All surface area (except "TERRAIN") will be treated in the same "area" dimension.
-> for Bureau surface area, we will take the smallest among all displayed area. 
(why? because advertised price often represent the minimum value product)
-> for pair like "m2" and "TERRAIN" will be treated as 2 scalars.
"""

"""
Now let's extract area, we know that area can be represented in 3 tags : 
-M²
-SURFAC
-TERRAIN
"""

def extract_area(df):
    """
    Normalize surface data into two scalars per row:
      - AREA: one scalar for all non-TERRAIN surfaces (or TERRAIN tokens without digits)
      - TERRAIN: one scalar for terrain (only if the token contains 'TERRAIN' + a digit)

    Inputs:
      df         : DataFrame
      col_seq    : column with a token list/tuple/set per row (used to detect presence of area tags)
      col_tag    : column with a token list/tuple/set per row (we extract numbers from here)
      area_tags  : {'M²','SURFAC','TERRAIN'}
      area_pattern: compiled regex to recognize any of M² / SURFAC(e) / TERRAIN tokens

    Returns:
      DataFrame copy with columns:
        - 'ALL AREA' : list of tokens recognized as area-related in col_tag
        - 'AREA'     : float (or <NA>)
        - 'TERRAIN'  : float (or <NA>)
    """
    area_tags = {"M²","SURFAC","TERRAIN"}
    has_area = df["CLEANED TAG"].apply(
        lambda xs: bool(set(xs) & set(area_tags)) if isinstance(xs, (list, tuple, set)) else False
    )
    out = df.loc[has_area].copy()
    def get_area_tokens(tokens):
        if not isinstance(tokens, (list, tuple, set)):
            return pd.NA
        hits = []
        for t in tokens:
            if re.search(PATTERN_AREA, str(t)):
                hits.append(str(t))
        if hits:
            return hits
        return pd.NA

    out["ALL AREA"] = out["DATA TAG"].apply(
        lambda xs: get_area_tokens(list(xs)) if isinstance(xs, (list, tuple, set)) else pd.NA
    )
    def get_lowest_value(tokens):
        if not tokens:
            return pd.NA
        vals = []
        for t in tokens:
            try:
                vals.extend(_numbers_in_text(t))
            except Exception:
                pass
        return min(vals) if vals else pd.NA

    def is_divisible(tokens):
        if not tokens:
            return pd.NA
        for t in tokens:
            if isinstance(t, str) and "DIVISIBL" in t:
                return True
        return False

    def split_area_vs_terrain(tokens):
        if not isinstance(tokens, (list, tuple, set)):
            return [], []
        area_tokens, terrain_tokens = [], []
        for t in tokens:
            t_str = str(t)
            has_terrain = bool(PATTERN_TERRAIN.search(t_str))
            has_digit   = bool(PATTERN_HAS_DIGIT.search(t_str))
            if has_terrain and has_digit:
                terrain_tokens.append(t_str)
            else:
                area_tokens.append(t_str)
        return area_tokens, terrain_tokens

    # 3) compute scalars for AREA and TERRAIN from 'ALL AREA'
    def compute_scalars(tokens):
        if not isinstance(tokens, (list, tuple, set)) or not tokens:
            return pd.Series({"AREA": pd.NA, "TERRAIN": pd.NA})

        area_tokens, terrain_tokens = split_area_vs_terrain(tokens)
        if area_tokens:
            if isinstance(is_divisible(area_tokens), bool) and is_divisible(area_tokens):
                area_val = get_lowest_value(area_tokens)
            else:
                area_val = get_lowest_value(area_tokens)
        else:
            area_val = pd.NA
        terrain_val = get_lowest_value(terrain_tokens) if terrain_tokens else pd.NA

        return pd.Series({"AREA": area_val, "TERRAIN": terrain_val})

    scalars = out["ALL AREA"].apply(compute_scalars)
    out.loc[:, "AREA"] = scalars["AREA"]
    out.loc[:, "TERRAIN"] = scalars["TERRAIN"]

    return out


In [225]:
#bool tag detect function
def is_dispo(tags):
    # DISPONIBL MAINTEN: 1 if present, 0 if explicitly absent, <NA> if no tokens
    if not isinstance(tags, (set, tuple, list)) or len(tags) == 0:
        return pd.NA
    return 1 if any(PATTERN_DISPO_NOW.search(str(t)) for t in tags) else 0

def is_neuf(tags):
    # NEUF: 1 if present, 0 if explicitly absent, <NA> if no tokens
    if not isinstance(tags, (set, tuple, list)) or len(tags) == 0:
        return pd.NA
    return 1 if any(PATTERN_NEUF.search(str(t)) for t in tags) else 0

def has_chambr(tags):
    # CHAMBR: 1 if present, <NA> otherwise (to be averaged later, per your design)
    if not isinstance(tags, (set, tuple, list)) or len(tags) == 0:
        return pd.NA
    return 1 if any(PATTERN_CHAMBRE.search(str(t)) for t in tags) else pd.NA

def has_piece(tags):
    # PIEC: 1 if present, <NA> otherwise (to be averaged later)
    if not isinstance(tags, (set, tuple, list)) or len(tags) == 0:
        return pd.NA
    return 1 if any(PATTERN_PIECE.search(str(t)) for t in tags) else pd.NA

def has_floor(tags):
    # ETAG/RDC: 1 if present, 0 if explicitly absent, <NA> if no tokens
    if not isinstance(tags, (set, tuple, list)) or len(tags) == 0:
        return pd.NA
    return 1 if any(PATTERN_FLOOR.search(str(t)) for t in tags) else 0

#bool for floor extraction
def is_rdc(tag):
    if not tag :
        raise ValueError("No tag provided")
    if bool(PATTERN_RDC.search(tag)):
        return 1
    else : return 0
def is_etag(tag):
    if not tag :
        raise ValueError("No tag provided")
    if bool(PATTERN_ETAG.search(tag)):
        return 1
    else : return 0    
def has_slash(tag):
    if not tag:
        raise ValueError("No tag provided")
    if bool(PATTERN_SLASH.search(tag)):
        return 1
    else : return 0

In [227]:
def extract_additionals(df,col_seq="CLEANED TAG",col_tag="DATA TAG"):
    #create masks
    mask_is_dispo = df[col_seq].apply(lambda x : is_dispo(list(x)) if isinstance(x,(tuple,list,set)) else pd.NA).fillna(0).astype(bool)
    mask_is_neuf = df[col_seq].apply(lambda x : is_neuf(list(x)) if isinstance(x,(tuple,list,set)) else pd.NA).fillna(0).astype(bool)
    mask_has_chambr = df[col_seq].apply(lambda x : has_chambr(list(x)) if isinstance(x,(tuple,list,set)) else pd.NA).fillna(0).astype(bool)
    mask_has_piece = df[col_seq].apply(lambda x : has_piece(list(x)) if isinstance(x,(tuple,list,set)) else pd.NA).fillna(0).astype(bool)
    mask_has_floor = df[col_seq].apply(lambda x : has_floor(list(x)) if isinstance(x,(tuple,list,set)) else pd.NA).fillna(0).astype(bool)
    #extract tags
    def get_chambr_tag(tags):
        if not tags :
            raise ValueError("Tags not provided")
        if not isinstance(tags,(set,list,tuple)):
            return pd.NA
        for t in tags :
            if bool(PATTERN_CHAMBRE.search(t)):
                return t
        return pd.NA
    def get_piece_tag(tags):
        if not tags :
            raise ValueError("Tags not provided")
        if not isinstance(tags,(set,list,tuple)):
            return pd.NA
        for t in tags :
            if bool(PATTERN_PIECE.search(t)):
                return t
        return pd.NA
    def get_floor_tag(tags):
        if not tags :
            raise ValueError("Tags not provided")
        if not isinstance(tags,(set,list,tuple)):
            return pd.NA
        for t in tags :
            if bool(PATTERN_FLOOR.search(t)):
                return t
        return pd.NA

    #get numerical values
    def get_floor(tag):
        if not tag :
            raise ValueError("No tag provided")
        nb_floor = PATTERN_BEFORE_SLASH.search(tag)
        if bool(nb_floor):
            return int(float(nb_floor.group(1)))
        else : return 0
    def get_floors(tag):
        if not tag :
            raise ValueError("No tag provided")
        nb_floors = PATTERN_AFTER_SLASH.search(tag)
        if bool(nb_floors):
            return int(float(nb_floors.group(1)))
        else : return 0
    def get_num(tag):
        tag = str(tag)
        if not tag :
            raise ValueError("tag not provided")
        num = PATTERN_NUM.search(tag)
        if bool(num):
            return int(num.group(0))
        else :
            return 0
    def split_floor_tag(tag):
        if not tag or not bool(PATTERN_FLOOR.search(tag)):
            raise ValueError("tag not provided or no floor pat in tag")
    
        is_rdc_flag = bool(PATTERN_RDC.search(tag))
        has_slash   = bool(PATTERN_SLASH.search(tag))
    
        floor, floors = 0, 0
        if is_rdc_flag:
            floor = 0
        if has_slash:
            nb_floor  = PATTERN_BEFORE_SLASH.search(tag)
            nb_floors = PATTERN_AFTER_SLASH.search(tag)
            if not is_rdc_flag and nb_floor:
                floor = int(float(nb_floor.group(1)))
            if nb_floors:
                floors = int(float(nb_floors.group(1)))
            if not nb_floors:
                floors = floor
        else:
            if not is_rdc_flag:
                floor = int(get_num(tag)) or 0
            floors = floor
    
        return (floor, floors)

    #bool extraction
    series_dispo = df[col_seq].apply(lambda x : is_dispo(x))
    series_neuf = df[col_seq].apply(lambda x : is_neuf(x))

    #unconditional extraction & pd.NA to be averaged
    #chambre
    series_chambr = df[col_seq].apply(lambda x : has_chambr(x))
    series_chambr.loc[series_chambr.notna()] = df.loc[mask_has_chambr,col_tag].apply(lambda x : get_chambr_tag(x))
    series_chambr.loc[series_chambr.notna()] = series_chambr.loc[series_chambr.notna()].apply(lambda x : int(get_num(x)) if x else pd.NA)
    is_valid_series_chambr = series_chambr.dropna().apply(lambda x : isinstance(x,int)).all()
    if not is_valid_series_chambr :
        raise ValueError("is_valid_series_chambr not valid")
    #piece
    series_piece = df[col_seq].apply(lambda x : has_piece(x))
    series_piece.loc[series_piece.notna()] = df.loc[mask_has_piece,col_tag].apply(lambda x : get_piece_tag(x))    
    series_piece.loc[series_piece.notna()] = series_piece.loc[series_piece.notna()].apply(lambda x : int(get_num(x)) if x else pd.NA)
    is_valid_series_piece = series_piece.dropna().apply(lambda x : isinstance(x,int)).all()
    if not is_valid_series_piece :
        raise ValueError("is_valid_series_piece not valid")
    #conditional extraction
    series_floor_tag = df[col_seq].apply(lambda x: has_floor(x))
    mask_floor_any = series_floor_tag.fillna(0).astype(bool)
    series_floor_tag.loc[mask_floor_any] = df.loc[mask_has_floor, col_tag].apply(lambda x: get_floor_tag(x) if x else pd.NA)
    series_floor_tag.loc[mask_floor_any] = series_floor_tag.loc[mask_floor_any].apply(lambda x: split_floor_tag(x) if isinstance(x, str) else (0, 0))
    idx = ~mask_floor_any
    series_floor_tag.loc[idx] = [(0, 0)] * int(idx.sum())

    
    # validate
    is_valid_series_floor = series_floor_tag.apply(lambda x: isinstance(x, tuple) and len(x) == 2).all()

    if not is_valid_series_floor :
        raise ValueError("is_valid_series_floor not valid")
    if len(series_floor_tag) == len(series_chambr) == len(series_piece) == len(series_dispo) == len(series_neuf) :
        out = df.copy(deep=True)
        out["DISPONIBL MAINTEN"] = series_dispo
        out["NEUF"] = series_neuf
        out["CHAMBRE"] = series_chambr
        out["PIECE"] = series_piece
        out[["FLOOR","FLOORS"]] = pd.DataFrame(series_floor_tag.tolist(), index=df.index)
        return out
    else : 
        raise ValueError("len don't match")

    

In [271]:
# ---------- helpers ----------
def round_int(arr):
    a = np.asarray(arr, dtype=float)
    return np.round(a).astype(int)

def as_float(arr):
    return np.asarray(arr, dtype=float)

def make_model(x_cols, alpha=1.0):
    pre = ColumnTransformer([
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("sc", StandardScaler()),
        ]), x_cols)
    ])
    return Pipeline([("pre", pre), ("reg", Ridge(alpha=alpha))])

# ---------- train three models (Option B) ----------
def train_option_b(df, alpha_piece=1.0, alpha_ch_direct=1.0, alpha_ch_chain=1.0):
    df = df.copy()
    for c in ["AREA", "PIECE", "CHAMBRE"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    df_piece    = df.dropna(subset=["PIECE"])
    df_ch_dir   = df.dropna(subset=["CHAMBRE"])
    df_ch_chain = df.dropna(subset=["CHAMBRE", "PIECE"])

    m_piece_from_area = make_model(["AREA"], alpha=alpha_piece)
    m_piece_from_area.fit(df_piece[["AREA"]], df_piece["PIECE"])

    m_chambre_from_area = make_model(["AREA"], alpha=alpha_ch_direct)
    m_chambre_from_area.fit(df_ch_dir[["AREA"]], df_ch_dir["CHAMBRE"])

    m_chambre_from_area_piece = make_model(["AREA", "PIECE"], alpha=alpha_ch_chain)
    m_chambre_from_area_piece.fit(df_ch_chain[["AREA", "PIECE"]], df_ch_chain["CHAMBRE"])

    return {
        "piece_from_area": m_piece_from_area,
        "chambre_from_area": m_chambre_from_area,
        "chambre_from_area_piece": m_chambre_from_area_piece,
    }

# ---------- predict from AREA only ----------
def predict_from_area(models, area_value: float):
    X_area = pd.DataFrame([{"AREA": float(area_value)}])

    # piece <- area
    piece_hat = float(models["piece_from_area"].predict(X_area)[0])
    piece_hat = max(1.0, piece_hat)
    piece_hat_round = int(round(piece_hat))

    # chambre direct <- area
    chambre_direct = float(models["chambre_from_area"].predict(X_area)[0])
    chambre_direct = max(0.0, chambre_direct)
    chambre_direct_round = int(round(min(piece_hat_round, chambre_direct)))

    # chambre chained <- (area, piecê)
    X_chain = pd.DataFrame([{"AREA": float(area_value), "PIECE": piece_hat}])
    chambre_chained = float(models["chambre_from_area_piece"].predict(X_chain)[0])
    chambre_chained = max(0.0, min(piece_hat, chambre_chained))
    chambre_chained_round = int(round(min(piece_hat_round, chambre_chained)))

    if chambre_direct_round == piece_hat_round :
        chambre_direct_round -= 1
        chambre_direct -= 1
    if chambre_chained_round == piece_hat_round :
        chambre_chained -= 1
        chambre_chained_round -= 1 

    return {
        "piece_hat": piece_hat,
        "piece_hat_round": piece_hat_round,
        "chambre_direct": chambre_direct,
        "chambre_direct_round": chambre_direct_round,
        "chambre_chained": chambre_chained,
        "chambre_chained_round": chambre_chained_round,
    }

# ---------- end-to-end: train + evaluate ----------
def train_and_evaluate_option_b(
    train_df: pd.DataFrame,
    eval_df: pd.DataFrame | None = None,
    alpha_piece: float = 1.0,
    alpha_ch_direct: float = 1.0,
    alpha_ch_chain: float = 1.0,
    test_size: float = 0.2,
    random_state: int = 42,
):
    # ensure numeric
    def to_num(df):
        out = df.copy()
        for c in ["AREA", "PIECE", "CHAMBRE"]:
            out[c] = pd.to_numeric(out[c], errors="coerce")
        return out

    if eval_df is None:
        tr_df, te_df = train_test_split(to_num(train_df), test_size=test_size, random_state=random_state)
    else:
        tr_df, te_df = to_num(train_df), to_num(eval_df)

    models = train_option_b(tr_df, alpha_piece, alpha_ch_direct, alpha_ch_chain)
    metrics = {}

    # PIECE <- AREA
    te_piece = te_df.dropna(subset=["PIECE"])
    if len(te_piece) > 0:
        p_pred = models["piece_from_area"].predict(te_piece[["AREA"]])
        p_pred = as_float(p_pred)

        p_mae = mean_absolute_error(te_piece["PIECE"], p_pred)
        p_r2  = r2_score(te_piece["PIECE"], p_pred)

        p_pred_pp    = np.maximum(1.0, p_pred)
        p_pred_round = round_int(p_pred_pp)

        piece_true_int = te_piece["PIECE"].astype(int).to_numpy()
        p_exact  = float(np.mean(p_pred_round == piece_true_int))
        p_within = float(np.mean(np.abs(p_pred_round - piece_true_int) <= 1))

        metrics["PIECE_from_AREA"] = {
            "MAE": float(p_mae), "R2": float(p_r2),
            "pct_exact": p_exact, "pct_within_1": p_within,
        }

    # CHAMBRE (direct) <- AREA
    te_ch = te_df.dropna(subset=["CHAMBRE"])
    if len(te_ch) > 0:
        c_pred_d = models["chambre_from_area"].predict(te_ch[["AREA"]])
        c_pred_d = as_float(c_pred_d)

        c_mae_d = mean_absolute_error(te_ch["CHAMBRE"], c_pred_d)
        c_r2_d  = r2_score(te_ch["CHAMBRE"], c_pred_d)

        piece_true = te_ch["PIECE"].to_numpy(dtype=float) if "PIECE" in te_ch.columns else np.full(len(te_ch), np.inf)

        c_pred_d_pp    = np.maximum(0.0, c_pred_d)
        c_pred_d_clip  = np.minimum(piece_true, c_pred_d_pp)
        c_pred_d_round = round_int(c_pred_d_clip)

        ch_true_int = te_ch["CHAMBRE"].astype(int).to_numpy()
        c_exact_d  = float(np.mean(c_pred_d_round == ch_true_int))
        c_within_d = float(np.mean(np.abs(c_pred_d_round - ch_true_int) <= 1))

        metrics["CHAMBRE_from_AREA (direct)"] = {
            "MAE": float(c_mae_d), "R2": float(c_r2_d),
            "pct_exact": c_exact_d, "pct_within_1": c_within_d,
        }

    # CHAMBRE (chained)
    te_ch_chain_true = te_df.dropna(subset=["CHAMBRE", "PIECE"])
    if len(te_ch_chain_true) > 0:
        # (a) True piece
        c_pred_ct = models["chambre_from_area_piece"].predict(te_ch_chain_true[["AREA", "PIECE"]])
        c_pred_ct = as_float(c_pred_ct)

        c_mae_ct = mean_absolute_error(te_ch_chain_true["CHAMBRE"], c_pred_ct)
        c_r2_ct  = r2_score(te_ch_chain_true["CHAMBRE"], c_pred_ct)

        c_pred_ct_pp    = np.maximum(0.0, np.minimum(te_ch_chain_true["PIECE"].to_numpy(dtype=float), c_pred_ct))
        c_pred_ct_round = round_int(c_pred_ct_pp)

        ch_true_int = te_ch_chain_true["CHAMBRE"].astype(int).to_numpy()
        c_exact_ct  = float(np.mean(c_pred_ct_round == ch_true_int))
        c_within_ct = float(np.mean(np.abs(c_pred_ct_round - ch_true_int) <= 1))

        metrics["CHAMBRE_from_(AREA,PIECE) (true piece)"] = {
            "MAE": float(c_mae_ct), "R2": float(c_r2_ct),
            "pct_exact": c_exact_ct, "pct_within_1": c_within_ct,
        }

        # (b) Area-only path: piecê first, then chambrê
        p_hat = models["piece_from_area"].predict(te_ch_chain_true[["AREA"]])
        p_hat = as_float(p_hat)
        p_hat_pp = np.maximum(1.0, p_hat)

        Xc = pd.DataFrame({"AREA": te_ch_chain_true["AREA"].values, "PIECE": p_hat_pp})
        c_pred_ca = models["chambre_from_area_piece"].predict(Xc)
        c_pred_ca = as_float(c_pred_ca)

        c_pred_ca_pp    = np.maximum(0.0, np.minimum(p_hat_pp, c_pred_ca))
        c_pred_ca_round = round_int(c_pred_ca_pp)

        c_mae_ca = mean_absolute_error(te_ch_chain_true["CHAMBRE"], c_pred_ca)
        c_r2_ca  = r2_score(te_ch_chain_true["CHAMBRE"], c_pred_ca)

        c_exact_ca  = float(np.mean(c_pred_ca_round == ch_true_int))
        c_within_ca = float(np.mean(np.abs(c_pred_ca_round - ch_true_int) <= 1))

        metrics["CHAMBRE_chained (AREA→PIECÊ→CHAMBRE)"] = {
            "MAE": float(c_mae_ca), "R2": float(c_r2_ca),
            "pct_exact": c_exact_ca, "pct_within_1": c_within_ca,
        }

    return models, metrics

In [261]:
def lin_predict_features(df,models):
    preds = {}
    mask_both_missing = df["PIECE"].isna() & df["CHAMBRE"].isna()
    mask_piece_missing = df["PIECE"].isna() & df["CHAMBRE"].notna()
    mask_chambre_missing = df["CHAMBRE"].isna() & df["PIECE"].notna()

    out = df.copy(deep=True)
    if mask_chambre_missing.any():
        X1 = out.loc[mask_chambre_missing,["AREA","PIECE"]].astype(float)
        out.loc[mask_chambre_missing, "CHAMBRE"]=np.round(
            np.maximum(0.0,np.minimum(
                X1["PIECE"].to_numpy(),
                models["chambre_from_area_piece"].predict(X1).astype(float)
                )
            )
        ).astype(int)

    if mask_both_missing.any():
        preds_by_area = out.loc[mask_both_missing,"AREA"].apply(lambda x : 
                                                                predict_from_area(models=models,area_value=float(x))
                                                               )
        out.loc[mask_both_missing,"PIECE"] = preds_by_area.apply(lambda x:x["piece_hat_round"] if isinstance(x,dict) else 0)
        out.loc[mask_both_missing,"CHAMBRE"] = preds_by_area.apply(lambda x:x["chambre_chained_round"] if isinstance(x,dict) else 0)

    if mask_piece_missing.any():
        out.loc[mask_piece_missing,"PIECE"] = out.loc[mask_piece_missing,"CHAMBRE"].astype(float) + 1         

    return out

In [509]:
def vectorize_dataset_seloger(csv_file : str = None):
    if not csv_file :
        raise ValueError("Require dataset filepath")
    #vopen file
    try :
        raw_df = pd.read_csv(csv_file)
    except Exception :
        raise ValueError("Error converting file.csv to dataframe")
    raw_df_cols = raw_df.columns.tolist()
    #verify columns
    if raw_df_cols != ['href', 'title']:
        raise ValueError(f"Unexpected columns {raw_df_cols}")

    
    #1 extract price,unit-price, data-tags -------------------------------
    out_df = get_processed(raw_df["title"]) 
    #2 clean tags (remove numbers, normalize them)
    out_df = out_df.loc[out_df["DATA TAG"].notna()]
    out_df["CLEANED TAG"] = out_df["DATA TAG"].apply(
        lambda x : normalize_tags(x) if isinstance(x,(tuple,list,set)) else pd.NA
    )
    out_df = out_df.loc[out_df["CLEANED TAG"].notna()]
    #3 extract type
    out_df["TYPE"] = out_df["CLEANED TAG"].apply(
        lambda x : extract_type(x) if isinstance(x,(tuple,list,set)) else pd.NA
    )
    out_df = out_df.loc[out_df["TYPE"].notna()]

    #4 extract location(postcode)
    out_df = extract_loc(df=out_df,dict_map=STEM_TO_POSTCODES)
    out_df = out_df.loc[out_df["LOC"].notna()]
    if out_df["LOC"].apply(lambda x : len(x) == 1 if isinstance(x,(set)) else False).any():
        out_df["LOC"] = out_df["LOC"].apply(lambda x : next(iter(x)))
    else :
        raise ValueError("Multiple location in one row detected.")
    #5 extract area,terrain
    out_df = extract_area(out_df)
    out_df = out_df.loc[~(out_df["AREA"].isna() & out_df["TERRAIN"].isna())]
    out_df.loc[out_df["TERRAIN"].isna(),"TERRAIN"] = 0
    out_df.loc[out_df["AREA"].isna(),"AREA"] = 0
    out_df["AREA"] = out_df["AREA"].astype(float)
    out_df["TERRAIN"] = out_df["TERRAIN"].astype(float)
    #extract feature
    out_df = extract_additionals(out_df)
    out_df["FLOOR"] = out_df["FLOOR"].astype(int)
    out_df["FLOORS"] = out_df["FLOORS"].astype(int)
    #fill missing values for chambre and piece, use linear regression model
    models,_ = train_and_evaluate_option_b(out_df.dropna())
    out_df = lin_predict_features(df=out_df,models=models)
    out_df["CHAMBRE"] = out_df["CHAMBRE"].astype(int)
    out_df["PIECE"] = out_df["PIECE"].astype(int)
    out_df.loc[out_df["TYPE"] == "TERRAIN CONSTRUCTIBL A VENDR" ,["PIECE","CHAMBRE","FLOOR","FLOORS"]] =0
    
    return out_df

In [511]:
vectorized_dataset = vectorize_dataset_seloger("data_sample_1584.csv")

C:\Users\thaim\AppData\Local\Temp\ipykernel_126624\3798004323.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask_has_chambr = df[col_seq].apply(lambda x : has_chambr(list(x)) if isinstance(x,(tuple,list,set)) else pd.NA).fillna(0).astype(bool)
C:\Users\thaim\AppData\Local\Temp\ipykernel_126624\3798004323.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask_has_piece = df[col_seq].apply(lambda x : has_piece(list(x)) if isinstance(x,(tuple,list,set)) else pd.NA).fillna(0).astype(bool)
C:\Users\thaim\AppData\Local\Temp\ipykernel_126624\3798004323.

#### Verify

In [542]:
vectorized_dataset.isna().sum()

RAW                  0
DATA TAG             0
PRICE                0
PRICE_UNIT           0
CLEANED TAG          0
TYPE                 0
LOC                  0
ALL AREA             0
AREA                 0
TERRAIN              0
DISPONIBL MAINTEN    0
NEUF                 0
CHAMBRE              0
PIECE                0
FLOOR                0
FLOORS               0
dtype: int64

In [547]:
vectorized_dataset.columns

Index(['RAW', 'DATA TAG', 'PRICE', 'PRICE_UNIT', 'CLEANED TAG', 'TYPE', 'LOC',
       'ALL AREA', 'AREA', 'TERRAIN', 'DISPONIBL MAINTEN', 'NEUF', 'CHAMBRE',
       'PIECE', 'FLOOR', 'FLOORS'],
      dtype='object')

In [575]:
"""
Verification for valid tags, there's some tags that's included in the output vectorized_dataset
Explanations : 
- DIVISIBLE was removed from filtering types process
- HOTEL PARTICULI A VENDR was later decided not to be used since our models focus on TERRAIN CONSTRUCTIBLE, MAISON, APPART (residential properties)
- TERRAIN is extracted to AREA,TERRAIN cols
"""
set(vectorized_dataset["CLEANED TAG"].explode().tolist()).symmetric_difference(valid_tokens)

{'DIVISIBL', 'HOTEL PARTICULI A VENDR', 'TERRAIN'}

In [579]:
#Verify type
set(vectorized_dataset["TYPE"].tolist())

{'APPART A VENDR',
 'DUPLEX A VENDR',
 'LOFT A VENDR',
 'MAISON A VENDR',
 'MAISON VILL A VENDR',
 'STUDIO A VENDR',
 'TERRAIN CONSTRUCTIBL A VENDR',
 'VILL A VENDR'}

In [633]:
#verify cols values
def get_max_val(df=vectorized_dataset,col=None):
    max_v = vectorized_dataset.loc[vectorized_dataset[col] == max(vectorized_dataset[col])]
    print(f"{col} : ",max_v[col].to_string(index = False))
    print("RAW : ",max_v["RAW"].iloc[0],"\n","*"*100)
check_cols = {"AREA","TERRAIN","CHAMBRE","PIECE","FLOOR","FLOORS","PRICE"}
for c in check_cols :
    get_max_val(col=c)

PIECE :  15
RAW :  Maison à vendre - Paris 17ème - 5 100 000 € - 15 pièces, 6 chambres, 500,3 m² 
 ****************************************************************************************************
FLOORS :  32
32
RAW :  Appartement à vendre - Paris 13ème - 549 000 € - 5 pièces, 3 chambres, 96 m², Étage 28/32 
 ****************************************************************************************************
FLOOR :  30
RAW :  Appartement à vendre - Paris 15ème - 410 000 € - 2 pièces, 1 chambre, 48 m², Étage 30/31 
 ****************************************************************************************************
AREA :  500.3
RAW :  Maison à vendre - Paris 17ème - 5 100 000 € - 15 pièces, 6 chambres, 500,3 m² 
 ****************************************************************************************************
TERRAIN :  499.0
RAW :  Maison de ville à vendre - Montreuil - 750 000 € - 8 pièces, 6 chambres, 240 m², 499 m² de terrain 
 **********************************************

In [643]:
def get_min_val(df=vectorized_dataset,col=None):
    max_v = vectorized_dataset.loc[vectorized_dataset[col] == min(vectorized_dataset[col])].iloc[0]
    print(f"{col} : ",max_v[col])
    print("RAW : ",max_v.iloc[0],"\n","*"*100)
#check_cols = {"AREA","TERRAIN","CHAMBRE","PIECE","FLOOR","FLOORS","PRICE"}
for c in check_cols :
    get_min_val(col=c)

PIECE :  0
RAW :  Terrain constructible à vendre - Montreuil - 450 000 € - 394 m² de terrain 
 ****************************************************************************************************
FLOORS :  0
RAW :  Duplex à vendre - Neuf - Saint-Cloud - 1 140 000 € - 4 pièces, 3 chambres, 100,8 m², RDC 
 ****************************************************************************************************
FLOOR :  0
RAW :  Appartement à vendre - Paris 18ème - 1 550 000 € - 5 pièces, 4 chambres, 191,9 m², RDC/1 
 ****************************************************************************************************
AREA :  0.0
RAW :  Terrain constructible à vendre - Montreuil - 450 000 € - 394 m² de terrain 
 ****************************************************************************************************
TERRAIN :  0.0
RAW :  Appartement à vendre - Paris 1er - 560 000 € - 2 pièces, 1 chambre, 45 m², Étage 5/6 
 *****************************************************************************

In [659]:
#do piece less than chambre ?
len(vectorized_dataset.loc[vectorized_dataset["PIECE"] < vectorized_dataset["CHAMBRE"]])

0

In [661]:
#seem good, let's save
vectorized_dataset.to_csv("vectorized_dataset_seloger_1584.csv",index=False)